# imports

In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#device = 0
#torch.cuda.set_device(device)

%reload_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

from typing import List, Callable

import sys

#https://github.com/FAU-DLM/wsi_processing_pipeline
sys.path.append("../")
sys.path.append("../wsi_processing_pipeline/")
sys.path.append("../wsi_processing_pipeline/tile_extraction")
import wsi_processing_pipeline
import tile_extraction
import preprocessing
from wsi_processing_pipeline.tile_extraction import tiles, util, slide, filter
from wsi_processing_pipeline.tile_extraction.tiles import WsiInfo

from wsi_processing_pipeline.preprocessing import *
import wsi_processing_pipeline.preprocessing.files_getter
import wsi_processing_pipeline.preprocessing.name_getter
from wsi_processing_pipeline.preprocessing.name_getter import NameGetter
from wsi_processing_pipeline.preprocessing.objects import ObjectManager

sys.path.append('../models-pytorch/pretrained-models.pytorch')
import pretrainedmodels
from pretrainedmodels import *


import fastai2
from typing import Dict
import pandas as pd
import numpy as np
import os
import torch
import torchvision
from torchvision.models import *
from torchsummary import summary
from functools import partial, update_wrapper
from tqdm import tqdm_notebook as tqdm
import matplotlib.image as mpimg
import shutil
from PIL import Image
import sklearn
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

import pathlib
from pathlib import Path
Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]

#against DecompressionBombWarning
Image.MAX_IMAGE_PIXELS = 10000000000   

PATH = Path('/home/Deep_Learner/private/datasets/Hypophysenadenome/')

WSIS_CORTICOTROP = PATH/'corticotrop'
WSIS_GONADOTROP = PATH/'gonadotrop'

ROIS_CORTICOTROP = PATH/'corticotrop_ROIs'
ROIS_GONADOTROP = PATH/'gonadotrop_ROIs'

TILES_CORTICOTROP_1 = PATH/'tiles_corticotrop_1_scoring_function_1_thresh_0.55'
TILES_CORTICOTROP_2 = PATH/'tiles_corticotrop_2_scoring_function_1_thresh_0.55'
TILES_CORTICOTROP_3 = PATH/'tiles_corticotrop_3_scoring_function_1_thresh_0.4'

TILES_GONADOTROP_1 = PATH/'tiles_gonadotrop_1_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_2 = PATH/'tiles_gonadotrop_2_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_3 = PATH/'tiles_gonadotrop_3_scoring_function_1_thresh_0.4'

#TEST = PATH/TEST_NAME
#TEST = PATH_LOCAL/TEST_NAME
TEST_EXPERIMENTING = PATH/'tiles_test_100_for_testing'
LABELS_CORTICOTROP_NAME = 'KortikotropHA_gelabled.xlsx'
LABELS_CORTICOTROP = PATH/LABELS_CORTICOTROP_NAME
LABELS_GONADOTROP_NAME = 'GonadotropeHA_gelabled.xlsx'
LABELS_GONADOTROP = PATH/LABELS_GONADOTROP_NAME
MODEL_PATH_NAME = 'models'
MODEL_PATH = PATH/MODEL_PATH_NAME

ROIS_EXPERIMENTING = PATH/'rois_experimenting'
TILES_EXPERIMENTING = PATH/'tiles_experimenting'

torch.backends.cudnn.benchmark=True


sz = 512
bs = 6


seed = 19
np.random.seed(seed)

num2lbs = {
    0:"corticotrop", 
    3:"silent",  
    8:"LH", 
    9:"FSH"
}

lbs2num = {l:n for n,l in num2lbs.items()}

# data preprocessing 

In [25]:
roi_paths=wsi_processing_pipeline.preprocessing.files_getter.FilesGetter().get_dirs_and_files(path=PATH, 
                                                                        get_files=True, 
                                                                        suffix='.png', 
                                                                        recursive=True, 
                                                                        folder_name=['gonadotrop_ROIs', 'corticotrop_ROIs'])



0it [00:00, ?it/s]

5it [00:05,  1.12s/it]

6it [00:05,  1.21it/s]

11it [00:06,  1.64it/s]

12it [00:12,  2.23s/it]

46it [00:15,  1.59s/it]

48it [00:15,  3.03it/s]


In [26]:
roi_paths = roi_paths[:5]

In [27]:
roi_paths.items[:5]

[Path('/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/393-13-Z-HE-ROI_18-LH+FSH.png'),
 Path('/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1536-15-Z-HE-ROI_32-LH.png'),
 Path('/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png'),
 Path('/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png'),
 Path('/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/105-15-Z-HE-ROI_27-LH+FSH.png')]

In [28]:
len(roi_paths.items)

5

In [59]:
len(ROIS_CORTICOTROP.ls() + ROIS_GONADOTROP.ls())

13169

In [30]:
def get_case_id_from_path(path):
    """
    Arguments:
        path: roi path
    """
    path = Path(path)
    split = path.stem.split('-')
    return f'{split[0]}-{split[1]}'

def get_slide_name_from_path(path):
    """
    Arguments:
        path: roi path
    """
    path = Path(path)
    split = path.stem.split('-')
    try:
        return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}'
    except IndexError:
        return f'{split[0]}-{split[1]}-{split[2]}'
    
df_c = pd.read_excel(LABELS_CORTICOTROP)
def label_func(path):
    """
    Arguments:
        path: roi path
    """
    path = Path(path)
    s = path.stem  
    if('LH+FSH' in s):
        return [lbs2num['LH'],lbs2num['FSH']]
    elif 'LH' in s:       
        return [lbs2num['LH']]
    elif 'FSH' in s:        
        return [lbs2num['FSH']]
    elif 'ACTH' in s:
        result = [lbs2num['corticotrop']]
        id = get_case_id_from_path(path)
        l = df_c.loc[df_c.id == id].label
        try:
            if str(lbs2num['silent']) in str(l.values[0]):
                result.append(3)
        except:
            print(l.values)
            print(s)
            print(get_slide_name_from_path(path))
            raise
        return result
    
def label_func_path_list(paths:List[pathlib.Path]):
    return [label_func(p) for p in paths]

In [31]:
roi_case_id_getter=NameGetter(func_path_to_case_id=get_case_id_from_path).get_name_from_path
roi_slide_id_getter=NameGetter(func_path_to_case_id=get_slide_name_from_path).get_name_from_path

In [32]:
named_objects=NameGetter().create_named_object_from_path(path=roi_paths.items, 
                                                   case_id_getter=roi_case_id_getter, 
                                                   slide_id_getter=roi_slide_id_getter,
                                                   classification_labels_getter=label_func_path_list)



5it [00:00, 64329.82it/s]

Patient ID has not been specified!


In [11]:
object_manager=ObjectManager(named_objects, splitter=sklearn.model_selection.train_test_split)

In [65]:
path_to_wsi_info = {}
for p in roi_paths.items:
    wsi_info = WsiInfo(path=p, 
                       patient_id=get_case_id_from_path, #one case per patient
                       case_id=get_case_id_from_path(p), 
                       slide_id=get_slide_name_from_path(p), 
                       classification_labels=label_func(p), 
                       dataset_type=None, 
                       rois=None)
    path_to_wsi_info[p] = wsi_info

In [66]:
tilesummaries = tiles.WsiOrROIToTilesMultithreaded(wsi_paths=roi_paths.items, 
                                                   tiles_folder_path=None, 
                                                   tile_height=512, 
                                                   tile_width=512, 
                                                   minimal_acceptable_tile_height=0.7, 
                                                   minimal_acceptable_tile_width=0.7, 
                                                   tile_score_thresh=0.3,
                                                   tile_scoring_function=tiles.scoring_function_1, 
                                                   level=0,
                                                   save_tiles=False, 
                                                   return_as_tilesummary_object=True, 
                                                   wsi_path_to_wsi_info=path_to_wsi_info)







  0%|          | 0/5 [00:00<?, ?it/s]





 20%|██        | 1/5 [00:08<00:32,  8.05s/it]





 40%|████      | 2/5 [00:08<00:17,  5.76s/it]





 60%|██████    | 3/5 [00:09<00:08,  4.44s/it]





 80%|████████  | 4/5 [00:11<00:03,  3.69s/it]





100%|██████████| 5/5 [00:44<00:00, 12.36s/it]

In [ ]:
for ts in tilesummaries:
    ts.show_wsi_with_marked_tiles()

In [68]:
object_manager.convert_to_wsi_or_roi_object_by_tilesummaries(tilesummaries=tilesummaries)

In [69]:
object_manager.export_dataframe()

,fname,labels,is_valid,patient_id,case_id,slide_id,dataset_type,path
0,"[Tile #170, Row #9, Column #2, Tissue 52.98%, Score 0.8672]",[9],None,<function get_case_id_from_path at 0x7f1831aaa320>,931-13,931-13-Z-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
1,"[Tile #169, Row #9, Column #1, Tissue 51.74%, Score 0.8583]",[9],None,<function get_case_id_from_path at 0x7f1831aaa320>,931-13,931-13-Z-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
2,"[Tile #213, Row #11, Column #3, Tissue 58.21%, Score 0.8580]",[9],None,<function get_case_id_from_path at 0x7f1831aaa320>,931-13,931-13-Z-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
3,"[Tile #3, Row #1, Column #3, Tissue 37.23%, Score 0.8537]",[9],None,<function get_case_id_from_path at 0x7f1831aaa320>,931-13,931-13-Z-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
4,"[Tile #67, Row #4, Column #4, Tissue 25.40%, Score 0.8502]",[9],None,<function get_case_id_from_path at 0x7f1831aaa320>,931-13,931-13-Z-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
...,...,...,...,...,...,...,...,...
349,"[Tile #42, Row #9, Column #2, Tissue 54.75%, Score 0.3181]","[8, 9]",None,<function get_case_id_from_path at 0x7f1831aaa320>,1048-12,1048-12-III-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png
350,"[Tile #1, Row #1, Column #1, Tissue 53.66%, Score 0.3096]","[8, 9]",None,<function get_case_id_from_path at 0x7f1831aaa320>,1048-12,1048-12-III-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png
351,"[Tile #28, Row #6, Column #3, Tissue 54.79%, Score 0.3092]","[8, 9]",None,<function get_case_id_from_path at 0x7f1831aaa320>,1048-12,1048-12-III-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png
352,"[Tile #36, Row #8, Column #1, Tissue 54.11%, Score 0.3091]","[8, 9]",None,<function get_case_id_from_path at 0x7f1831aaa320>,1048-12,1048-12-III-HE,None,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png


In [73]:
object_manager.split(test_size=0.2, train_size=None, random_state=seed, shuffle=True, stratify=None)

In [74]:
object_manager.export_dataframe()

,fname,labels,is_valid,patient_id,case_id,slide_id,dataset_type,path
0,"[Tile #170, Row #9, Column #2, Tissue 52.98%, Score 0.8672]",[9],False,None,931-13,931-13-Z-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
1,"[Tile #169, Row #9, Column #1, Tissue 51.74%, Score 0.8583]",[9],False,None,931-13,931-13-Z-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
2,"[Tile #213, Row #11, Column #3, Tissue 58.21%, Score 0.8580]",[9],False,None,931-13,931-13-Z-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
3,"[Tile #3, Row #1, Column #3, Tissue 37.23%, Score 0.8537]",[9],False,None,931-13,931-13-Z-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
4,"[Tile #67, Row #4, Column #4, Tissue 25.40%, Score 0.8502]",[9],False,None,931-13,931-13-Z-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/931-13-Z-HE-ROI_16-FSH.png
...,...,...,...,...,...,...,...,...
349,"[Tile #42, Row #9, Column #2, Tissue 54.75%, Score 0.3181]","[8, 9]",False,None,1048-12,1048-12-III-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png
350,"[Tile #1, Row #1, Column #1, Tissue 53.66%, Score 0.3096]","[8, 9]",False,None,1048-12,1048-12-III-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png
351,"[Tile #28, Row #6, Column #3, Tissue 54.79%, Score 0.3092]","[8, 9]",False,None,1048-12,1048-12-III-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png
352,"[Tile #36, Row #8, Column #1, Tissue 54.11%, Score 0.3091]","[8, 9]",False,None,1048-12,1048-12-III-HE,DatasetType.train,/home/Deep_Learner/private/datasets/Hypophysenadenome/gonadotrop_ROIs/1048-12-III-HE-ROI_16-LH+FSH.png
